In [12]:
import os
ON_KAGGLE = os.path.exists('/kaggle/input')
ENABLE_SUBMISSION = True
GENERATE_WHL = False
os.environ["ON_KAGGLE"] = str(ON_KAGGLE)
env = None

In [13]:
# Install required packages if on Kaggle
if ON_KAGGLE:
    !pip install -U /kaggle/input/accelerate-wheelwhl/accelerate-0.29.1-py3-none-any.whl -qq
    !pip install -U /kaggle/input/bitsandbytes-0-42-0-py3-none-any-whl/bitsandbytes-0.42.0-py3-none-any.whl -qq
    !pip install -U /kaggle/input/omegaconf222py3/antlr4_python3_runtime-4.9.3-py3-none-any.whl -qq
    !pip install -U /kaggle/input/omegaconf222py3/omegaconf-2.2.2-py3-none-any.whl -qq
    !pip install -U /kaggle/input/hydracore120py3/hydra_core-1.2.0-py3-none-any.whl -qq
    !pip install -U /kaggle/input/parglare-0-18-0/parglare-0.18.0-py3-none-any.whl -qq
    !pip install -U /kaggle/input/rank-bm25-whl/rank_bm25-0.2.2-py3-none-any.whl -qq
    !pip install -U /kaggle/input/huggingface-trl-package/shtab-1.7.1-py3-none-any.whl -qq
    !pip install -U /kaggle/input/huggingface-trl-package/tyro-0.7.3-py3-none-any.whl -qq
    !pip install -U /kaggle/input/huggingface-trl-package/trl-0.7.11-py3-none-any.whl -qq
    !pip install -U /kaggle/input/amino-gaz-whl/aimo_gaz-0.0.1-py3-none-any.whl -qq --no-deps --force-reinstall
else:
    !pip install -r requirements.txt
    !pip install -e . --force-reinstall
if GENERATE_WHL:
    !pip install --upgrade build
    !python -m build

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [14]:
import time
from aimo_gaz.solver.solver_config import parse_solver_config
from aimo_gaz.tools.log_utils import setup_logger
from hydra import compose, initialize

config_path = '/kaggle/input/aimo-gaz-configs/configs' if ON_KAGGLE else 'src/aimo_gaz/configs'
config_name = 'coordination_solver_config.yaml'

# Setup the logger
if "AIMO_GAZ_ROOT" not in os.environ:
    os.environ["AIMO_GAZ_ROOT"] = '/kaggle/working' if ON_KAGGLE else 'src/aimo_gaz'
log_dir_name = "logs" if ON_KAGGLE else '.logs'
root_dir = os.environ["AIMO_GAZ_ROOT"]
# Check if the configs exists in the root directory
if not os.path.exists(os.path.join(root_dir, 'configs')):
    os.makedirs(os.path.join(root_dir, 'configs'))
# Copy the configs to the root directory
os.system(f'cp -r {config_path}/* {os.path.join(root_dir, "configs")}')
# Create the logs directory
if not os.path.exists(os.path.join(root_dir, log_dir_name)):
    os.makedirs(os.path.join(root_dir, log_dir_name), exist_ok=True)
time_str = time.strftime("%Y-%m-%d_%H-%M-%S")
os.makedirs(os.path.join(root_dir, log_dir_name, time_str), exist_ok=True)
logger = setup_logger("aimo_gaz", os.path.join(root_dir, log_dir_name, time_str, 'aimo_gaz.log'))
logger.info(f"Using config file: {config_name}")
with initialize(version_base="1.2", config_path="configs", job_name="aimo_gaz"):
    cfg = compose(config_name=config_name) # Load the config file
    solver_config = parse_solver_config(cfg)
solver = solver_config.get_solver(logger=logger)
assert solver is not None, "Solver not initialized"
    

/opt/conda/lib/python3.10/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'planner_solver_config.yaml': Defaults list is missing `_self_`. See https://hydra.cc/docs/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)
/opt/conda/lib/python3.10/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'execution_solver_config.yaml': Defaults list is missing `_self_`. See https://hydra.cc/docs/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)
/opt/conda/lib/python3.10/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'code_solver_config.yaml': Defaults list is missing `_self_`. See https://hydra.cc/docs/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)


In [15]:
# Set up the evaluation API
if ENABLE_SUBMISSION and ON_KAGGLE:
    import aimo
    if env is None:
        env = aimo.make_env()
    iter_test = env.iter_test()

In [16]:
if ENABLE_SUBMISSION and ON_KAGGLE:
    with solver:
        # Iterate through the test set and use the model make predictions
        for test, sample_submission in iter_test:
            print("Problem statement:\n")
            problem_statement = test['problem'][0]
            print(problem_statement)
            sample_submission['answer'] = solver.solve(problem_statement)
            env.predict(sample_submission)
            print("Submission:\n")
            print(sample_submission, '\n')
            print("="*50)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Problem statement:

What is $1-1$?
Submission:

       id  answer
0  000aaa       0 

Problem statement:

What is $0\times10$?
Submission:

       id  answer
0  111bbb       0 

Problem statement:

Solve $4+x=4$ for $x$.
Submission:

       id  answer
0  222ccc       0 

